In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time # for sleep time
import csv
import re
import functools
import datetime #for csv file title
import pandas as pd

## Get the Chicago hotel list from Hotels.com

#### Reference: https://github.com/priyasrivast/WebscrapingAirBnbAndHotels/blob/master/selenium_code/hotelOrl.ipynb

In [36]:
#open the hotel list page for Chicago area
driver = webdriver.Chrome(executable_path=r'C:\Users\Xitong Hu\Desktop\Class\BAN5193\chromedriver.exe')
driver.get('https://www.hotels.com/search.do?resolved-location=CITY%3A1497539%3AUNKNOWN%3AUNKNOWN&destination-id=1497539&q-destination=Chicago,%20Illinois,%20United%20States%20of%20America&q-check-in=2020-10-26&q-check-out=2020-10-27&q-rooms=1&q-room-0-adults=2&q-room-0-children=0')

In [37]:
#define functions for scrapping the hotel list. 
#We need to scroll down to the bottom of the page, to get more hotel lists.
#so we are going to design a program to scroll the page down automatically until it reached the bottom of the list. 
#The method is referenced from a Github repository
#https://github.com/priyasrivast/WebscrapingAirBnbAndHotels/blob/master/selenium_code/hotelOrl.ipynb

def reduce_concat(x, sep=""):
    return functools.reduce(lambda x, y: str(x) + sep + str(y), x)

def paste(*lists, sep=" ", collapse=None):
    result = map(lambda x: reduce_concat(x, sep=sep), zip(*lists))
    if collapse is not None:
        return reduce_concat(result, sep=collapse)
    return list(result)

def scroll_bar():
    #scroll to end of page so that all elements are visible
    calm = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//html"))) #if elements aren't available it will return a timeout error and alert user that script must be altered
    time.sleep(3)
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False

    while(match==False):
        lastCount = lenOfPage
        time.sleep(3)
        lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount==lenOfPage:
            match=True

    scroll = driver.find_element_by_xpath('//html')
    time.sleep(2)
    scroll.send_keys(Keys.HOME)
    time.sleep(2)

In [38]:
#Start to scroll down the hotel list page and detect how many hotel blocks (object) in the list page.
wait_event = WebDriverWait(driver, 10)
events = wait_event.until(EC.presence_of_all_elements_located((By.XPATH,'//div[@class="description resp-module"]')))
lenOfPage = driver.execute_script("window.scrollTo(0, (document.body.scrollHeight));var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    
hotel_all_links = []
newPos = 0
while (newPos <= lenOfPage):
    print('Number of events:', len(events))
    print('position pre-loop:', newPos) #for testing
    scroll_bar()
    time.sleep(3)

    offprep = paste(["var offprep = window.screen.height * "], [(len(events))/5], [";return offprep"], sep='') 
    offprep = ''.join(offprep)
    offset = driver.execute_script(offprep)
    newPos = newPos + offset
    scrollerArg = paste(["window.scrollTo(0, "], [newPos], [");"], sep = '')
    scrollerArg = ''.join(scrollerArg)
    time.sleep(20)
    scroller = driver.execute_script(scrollerArg)
    events = wait_event.until(EC.presence_of_all_elements_located((By.XPATH,'//div[@class="description resp-module"]')))

    print('Number of events:', len(events)) #testing
    print('current position:', newPos) #testing


Number of events: 14
position pre-loop: 0
Number of events: 466
current position: 3024
Number of events: 466
position pre-loop: 3024
Number of events: 466
current position: 103680


In [39]:
# Get list of links to hotels and append   
time.sleep(40)
hotel_list = (driver.find_elements_by_xpath('//div[@class="description resp-module"]/figure[@class="image"]/a'))
hotel_all_links = [hotel.get_attribute('href') for hotel in hotel_list ] 

#export the URLs to a csv file
df_links=pd.DataFrame(hotel_all_links,columns=['url'])
df_links.to_csv("Chicago_hotel_list.csv",index=False)